Vocab

In [6]:
import pandas as pd
import numpy as  np
from gensim.models import Word2Vec
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, random_split
import pickle

In [ ]:
df_title = pd.read_pickle('./data/clean_title.pkl')
df_text = pd.read_pickle('./data/clean_text.pkl')
print(df_title.columns)
print(df_text.columns)
print(df_title.head())
print(df_text.head())

In [ ]:
model_text = Word2Vec(df_text["text"], vector_size=100)
model_title = Word2Vec(df_title["title"], vector_size=100)
model_text.save("./data/model_text")
model_title.save("./data/model_title")
print(model_title.corpus_total_words)

In [7]:
model_text = Word2Vec.load("./data/model_text")
model_title = Word2Vec.load("./data/model_title")

In [16]:
# test with smaller amount of data
FRACTION = 0.01
size = int(df_text.shape[0] * FRACTION)
print(size)


# make tensors
def text_to_tensor(text_list, word_model):
    vectors = [word_model.wv[word] for word in text_list if word in word_model.wv]
    return torch.tensor(vectors)


def stack_tensor(df: pd.DataFrame, col_name, word_model):
    tensor_ls = []
    label_ls = []
    for row in df.itertuples(index=False):
        # print(row[df.columns.get_loc('class')])
        # print(row[df.columns.get_loc(col_name)])
        label_ls.append(row[df.columns.get_loc("class")])
        tensor_ls.append(text_to_tensor(row[df.columns.get_loc(col_name)], word_model))
    padded_ls = pad_sequence(tensor_ls, batch_first=True)
    return torch.tensor(label_ls), torch.stack(tuple(padded_ls))


text_labels_tensor, text_tensor = stack_tensor(
    df_text.iloc[:size, :], 
    col_name="text", 
    word_model=model_text
)
print(text_tensor.size())
print(text_labels_tensor.size())
text_dataset = TensorDataset(text_tensor, text_labels_tensor)

713
torch.Size([713, 12202, 100])
torch.Size([713])


In [24]:
def split(tensor_dataset, test_split=0.2):
    test_size = int(len(tensor_dataset) * test_split)
    train_size = int(len(tensor_dataset) - test_size)
    print(test_size)
    print(train_size)
    train_data, test_data = random_split(tensor_dataset, [train_size, test_size])
    return train_data, test_data

train_text_tensor, test_text_tensor = split(text_dataset,0.2)

with open("./data/train_text","wb") as f:
    pickle.dump(train_text_tensor,f)
with open("./data/test_text","wb") as f:
    pickle.dump(train_text_tensor,f)


142
571
